<a href="https://colab.research.google.com/github/lee7py/2021-py-DNN/blob/main/code/4-5%20my_beginner_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [ ]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [ ]:
# 뉴런 결과가 softmax를 취한 확률 값이 아닌 경우, from_logits=True
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

target = [[0], [1], [2]] 
output = [[1.2, 0.3, 0.1], [.15, .89, .26], [.25, .41, .94]]

loss = cce(target, output)
print(loss.numpy())

In [ ]:
cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)

target = [[0], [1], [2]] 
output = [[1.0, 0., 0.], [.05, .89, .06], [.05, .01, .94]]

loss = cce(target, output)
print(loss.numpy())

In [ ]:
cce = tf.keras.losses.CategoricalCrossentropy(from_logits = True)

target = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]] 
output = [[1.2, 0.3, 0.1], [.15, .89, .26], [.25, .41, .94]]

loss = cce(target, output)
print(loss.numpy())

In [ ]:
cce = tf.keras.losses.CategoricalCrossentropy(from_logits = False)

target = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]] 
output = [[1.0, 0., 0.], [.05, .89, .06], [.05, .01, .94]]

loss = cce(target, output)
print(loss.numpy())

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
print(y_train[0])

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

In [ ]:
tf.math.argmax(predictions, 1).numpy()

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
#학습 데이터 첫 번째의 결과 확률
sm = tf.nn.softmax(predictions)
sm.numpy()

In [ ]:
tf.reduce_sum(sm).numpy()

In [ ]:
max = tf.math.reduce_max(sm).numpy()
print(max)

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

In [ ]:
sm = sm.numpy()

print(-tf.math.log(sm[0][y_train[0]]).numpy())
# print(-tf.math.log(max).numpy())

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [ ]:
# 다음 값도 같다!
tf.keras.losses.sparse_categorical_crossentropy(y_train[:1], sm).numpy()

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

In [ ]:
predictions = model(x_test[:1]).numpy()
# predictions = model(x_test[1:2]).numpy()
sm = tf.nn.softmax(predictions)
sm.numpy()

from_tensor_slices()

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(np.arange(5))
for subset in ds:
    print(subset)

In [ ]:
import numpy as np
ds = tf.data.Dataset.from_tensor_slices((np.arange(10), np.arange(10, 20))).shuffle(5).batch(3)

for subset1, subset2 in ds:
    print(subset1)
    print(subset2)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(np.arange(5)).batch(3)
i = 0
for subset in ds:
    i += 1
print(i)

# 직접 구현한 방식(전문가 방식)

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

print(tf.__version__, tf.executing_eagerly())

In [ ]:
##################################################################################
# ① 문제와 정답 데이터 지정
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
##################################################################################
# tf.data를 사용해 훈련 데이터와 검증 데이터를 나눔
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
##################################################################################
# ② 인공신경망 모델 생성(구성)
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
##################################################################################
# ③ 학습에 필요한 최적화 방법과 손실 함수 등 지정
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

##################################################################################
# ④ 생성된 모델로 훈련 데이터 학습
# 손실과 정확도를 위한 객체
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
##################################################################################
# GradientTape()로 모델을 훈련하는 함수
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
# 모델을 테스트 또는 검증하기 위한 함수
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
# 실제 전체 훈련 데이터의 5회를 훈련
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  # template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  template = '{} Epoch 손실={:.4f}, 정확도={:.4f}, 검증손실={:.4f}, 검정정확도={:.4f}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

In [ ]:
##################################################################################
# ⑤ 테스트 데이터로 성능 평가
test_step(x_test, y_test)
print('테스트 데이터 정확도(Accuracy): %.4f' % (test_accuracy.result() * 100))